In [1]:
import torch
import math
import chime
from data_loader import NEFG3x3Set
from LVAE import LVAE

import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output
if torch.cuda.is_available():  
  dev = "cpu" 
else:  
  dev = "cpu"  

device = torch.device(dev) 
torch.cuda.set_device(0) 
# Download the MNIST Dataset
dataset = NEFG3x3Set("info_dat_charge.csv",
                     "data/3x12_16_damp00", "dat_charge", transform=True,device="cpu")

length = len(dataset)
train_split = math.floor(length*.7)
test_split = length - train_split

train_inds, test_inds = torch.utils.data.random_split(
    dataset, [train_split, test_split], generator=torch.Generator().manual_seed(42))


# Model Initialization
model = LVAE(1, [16, 16, 16, 16], [8, 16, 32, 64], kernel=5,  padding=0, stride=1)

# Validation using MSE Loss function
# loss_function = torch.nn.L1Loss()

# Using an Adam Optimizer with lr = 0.
optimizer = torch.optim.SGD(model.parameters(),
                             lr=1e-3)

train_data = torch.utils.data.DataLoader(dataset=train_inds, batch_size=32,
                                         shuffle=True)
test_data = torch.utils.data.DataLoader(dataset=test_inds, batch_size=32,
                                        shuffle=True)


In [2]:
epochs = 10
outputs = []
losses = []
local_loss  = 0
txt1 = "Epoch {epoch} / "+str(epochs)
txt2 = "Average training loss: {loss}"
txt3 = "Average test loss: {loss}"


for epoch in range(epochs):
    if epoch%10==0:
        clear_output(wait=True)
    print(txt1.format(epoch = epoch))
    for (inp,_ , tar, _,_,_) in train_data:
        model.train()
        inp = inp.float()
        tar = tar.float()
        # Output of Autoencoder
        rec = model(tar)
        # print(rec[1])

        # plt.imshow(reconstructed[0].cpu().detach().numpy().reshape(71,-1))
        # plt.show()
        # Calculating the loss function
        loss = model.loss_function(rec[0],rec[1], rec[2],M_N=1)
        # print(loss['loss'])
        local_loss=local_loss+loss['loss']
        optimizer.zero_grad()
        loss['loss'].backward()
        optimizer.step()
        
    
    print(txt2.format(loss = local_loss/len(train_data)))
    losses.append(local_loss/len(test_data))

    local_loss = 0
    kl_loss = 0
    for (inp, cmpr,tar, _,_,_) in test_data:
        model.eval()
        
        inp = inp.float()
        tar = tar.float()
        
        # Output of Autoencoder
        rec = model(tar)

        # Calculating the loss function
        loss = model.loss_function(rec[0],rec[1], rec[2],M_N=1)



        local_loss=local_loss+loss['loss'].item()
        kl_loss+=loss['KLD'].item()

    
    print(txt3.format(loss = local_loss/len(test_data)))
    print(kl_loss/len(test_data))
    
    
    local_loss = 0

chime.success()


Epoch 0 / 10
Average training loss: 0.8083544969558716
Average test loss: 0.32579366071149707
-0.07448126369854435
Epoch 1 / 10
Average training loss: 0.1203489601612091


In [ ]:
# Defining the Plot Style
plt.style.use('fivethirtyeight')
plt.xlabel('Epochs')
plt.ylabel('Loss')
 
# Plotting the last 100 values
a = [i.detach().numpy() for i in losses]

plt.plot(a)
plt.show()

In [ ]:
Lf = torch.nn.MSELoss()

loss_og =0
loss_cmp=0
loss_inp=0
test_data = torch.utils.data.DataLoader(dataset=test_inds, batch_size=1,
                                        shuffle=True)
for (inp, cmpr,tar, _,_,_) in test_data:
    
    inp = inp.reshape(1,1,71,26).float()
    cmpr = cmpr.reshape(1,1,71,26).float()
    tar = tar.reshape(1, 1,71,26).float()
    # print(inp.shape)
    rec = model(tar)[0].float().reshape(1, 1,71, 26)
    
    loss_inp += Lf(tar, inp).item()
    loss_cmp += Lf(tar, cmpr).item()
    loss_og += Lf(tar, rec).item()
    
print(loss_inp/len(dataset))
print(loss_og/len(dataset))
print(loss_cmp/len(dataset))


In [ ]:
torch.save(model.state_dict(), "trained_models/conv")

# Notes

Do a full deivice
